In [1]:
import anthropic
import os

In [2]:
ANTHROPIC_API_KEY = os.getenv(
    "ANTHROPIC_API_KEY"
)

client = anthropic.Anthropic(
    api_key=ANTHROPIC_API_KEY
)

def run_structured_data_converter(html_data):
    message = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=1024,
        system="You are a season web scraper, creating structured data from HTML.",
        messages=[
            {"role": "user", "content": f"Please convert the HTML provided below into structured JSON output, containing the most important elements only. #HTML/n{html_data}"},
        ]
    )

    return message.content

In [8]:
import requests
from bs4 import BeautifulSoup

def get_html_content(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes
        return response.text
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        return None

def scrape_ebay_product(url):
    try:
        html_content = get_html_content(url)
        if not html_content:
            return None
        
        soup = BeautifulSoup(html_content, 'html.parser')
        
        product_info = {}
        
        # Extract title
        title_elem = soup.find('h1', {'class': 'x-item-title__mainTitle'})
        if title_elem:
            product_info['title'] = title_elem.text.strip()
        
        # Extract price
        price_elem = soup.find('div', {'class': 'x-price-primary'})
        if price_elem:
            product_info['price'] = price_elem.text.strip()
        
        # Extract condition
        condition_elem = soup.find('div', {'class': 'x-item-condition-text'})
        if condition_elem:
            product_info['condition'] = condition_elem.text.strip()
        
        # Extract item specifics
        item_specifics = {}
        specifics_container = soup.find('div', {'class': 'ux-layout-section__item--table-view'})
        if specifics_container:
            for row in specifics_container.find_all('div', {'class': 'ux-layout-section__row'}):
                label = row.find('div', {'class': 'ux-labels-values__labels'})
                value = row.find('div', {'class': 'ux-labels-values__values'})
                if label and value:
                    item_specifics[label.text.strip()] = value.text.strip()
        product_info['item_specifics'] = item_specifics
        
        # Extract description
        description_iframe = soup.find('iframe', {'id': 'desc_ifr'})
        if description_iframe:
            desc_url = description_iframe.get('src')
            if desc_url:
                desc_html = get_html_content(desc_url)
                if desc_html:
                    desc_soup = BeautifulSoup(desc_html, 'html.parser')
                    description = desc_soup.get_text(strip=True)
                    # Trim description to first 500 characters
                    product_info['description'] = description[:500] + ('...' if len(description) > 500 else '')
        
        return product_info
    
    except Exception as e:
        print(f"An error occurred while scraping: {e}")
        return None


# Example usage
url = "https://www.ebay.com/itm/383841870626" # RoboVac product (random product i picked)
roboVac_html = scrape_ebay_product(url)
if roboVac_html:
    for key, value in roboVac_html.items():
        print(f"{key}: {value}")
else:
    print("Failed to retrieve product information")

title: eufy RoboVac 25C Robotic Vacuum Cleaner Wi-Fi Smart Automatic Sweeper Robot APP
price: US $69.99
condition: Certified - RefurbishedCertified - RefurbishedMore information - About this item condition
item_specifics: {}
description: eBayEufy RoboVac 25C Wi-Fi Connected Robot Vacuum|eBay Certified RefurbishedProduct DetailsRoboVac 25CThe Super-Slim and Even More Powerful RoboVac with Wi-Fi ConnectivityFrom eufy, the brand that lights up your life•Smart technology and innovative design that make your life easier• Created by the same team as Anker—renowned for exceptional quality and innovationLean, Mean, Cleaning MachineRoboVac may be super-slim, but it’s packed with a 3-point cleaning system, BoostIQ™ technology, and up to 1...
